In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv", sep=',')

In [3]:
def filterBoard(board):
    if  "cbse" in board.lower():
        return "cbse"
    elif "central board" in board.lower():
        return "cbse"
    elif "icse" in board.lower():
        return "icse"
    elif board == "0":
        return "na"
    else:
        return "state"

def getSpecialisationStats(series):
    dict = {}
    for index, value in series.iteritems():
        list = value.split(" ")
        for str in list:
            if str in dict:
                dict[str] += 1
            else:
                dict[str] = 1
    return dict

def addCustomSpecialisationColumns(dataframe):
    series = dataframe['Specialization']
    searchwords = ["computer","mechanical","biotechnology","science","applied","instrumentation","electronics","electrical","application","production","information","eng","telecommunications","technology","civil","industrial","control","engineering","other"]
    customColumns = {"computer":[],"mechanical":[],"biotechnology":[],"science":[],"applied":[],"instrumentation":[],"electronics":[],"electrical":[],"application":[],"production":[],"information":[],"eng":[],"telecommunications":[],"technology":[],"civil":[],"industrial":[],"control":[],"engineering":[], "other":[]}
    # For each row in the column,
    for row in dataframe['Specialization']:
        keywords = row.lower().split(" ")
        tempmap = {"computer":False,"mechanical":False,"biotechnology":False,"science":False,"applied":False,"instrumentation":False,"electronics":False,"electrical":False,"application":False,"production":False,"information":False,"eng":False,"telecommunications":False,"technology":False,"civil":False,"industrial":False,"control":False,"engineering":False,"other":False}
        someKeyWasFound = False
        for word in keywords:
            if word in searchwords:
                if word == "eng":
                    customColumns["engineering"].append(1)
                    tempmap["engineering"] = True
                    someKeyWasFound = True
                else:
                    customColumns[word].append(1)
                    tempmap[word] = True
                    someKeyWasFound = True

        if someKeyWasFound == False:
            customColumns["other"].append(1)
            tempmap["other"]=True

        # print len(tempmap)
        for key,value in tempmap.iteritems():
            if value is False:
                customColumns[key].append(0)


    # this should all be of same length
    for key, list in customColumns.iteritems():
        # print len(list)
        if key!="eng":
            dataframe[key] = list

    return searchwords


def modifyScoreColumns(dataframe):
    columns = ["Domain",	"ComputerProgramming",	"ElectronicsAndSemicon"	,"ComputerScience", 	"MechanicalEngg"	,"ElectricalEngg",	"TelecomEngg",	"CivilEngg"]
    for i, row in dataframe.iterrows():
        for column in columns:
            value = row[column]
            if value == -1:
                dataframe.set_value(i,column,0)
    return columns


In [4]:
def main():
    T = 32
    df = pd.read_csv("train.csv", sep=',')
    # city = df['JobCity']
    # distinctCities = np.sort(city.unique())
    # print df['Gender'].unique()
    # print df['CollegeTier'].unique()
    #print df['Degree'].unique()
    # print df['CollegeState'].unique()
    # print df['CollegeCityTier'].unique()
    # df = df.tail()

    dumCompanyName = pd.get_dummies(df['Gender'])
    result = dumCompanyName

    df['10board'] = df['10board'].apply(filterBoard)
    df['12board'] = df['12board'].apply(filterBoard)
    dataframe = df['10board']
    dummyFrame = pd.get_dummies(dataframe)
    result = pd.concat([result, dummyFrame], axis=1)

    dataframe = df['12board']
    dummyFrame = pd.get_dummies(dataframe)
    result = pd.concat([result, dummyFrame], axis=1)

    dataframe = df['Degree']
    dummyFrame = pd.get_dummies(dataframe)
    result = pd.concat([result, dummyFrame], axis=1)

    searchwords = addCustomSpecialisationColumns(df)
    for word in searchwords:
        if word!="eng":
            dummyFrame = df[word]
            result = pd.concat([result, dummyFrame], axis=1)

    dataframe =df['CollegeState']
    dummyFrame = pd.get_dummies(dataframe)
    result = pd.concat([result, dummyFrame], axis=1)

    yearConverter = lambda x: x.year
    df['myDOB'] = pd.to_datetime(df['DOB'])
    df['myDOB'] = df['myDOB'].apply(yearConverter)
    df['12age'] = df['12graduation'] - df['myDOB']
    df['gradage'] = df['GraduationYear'] - df['myDOB']

    result = pd.concat([result, df['myDOB']], axis=1)
    result = pd.concat([result, df['12age']], axis=1)
    result = pd.concat([result, df['gradage']], axis=1)
    result = pd.concat([result, df['GraduationYear']], axis=1)
    result = pd.concat([result, df['12graduation']], axis=1)

    result = pd.concat([result, df['10percentage']], axis=1)
    result = pd.concat([result, df['12percentage']], axis=1)
    result = pd.concat([result, df['CollegeTier']], axis=1)
    result = pd.concat([result, df['collegeGPA']], axis=1)
    result = pd.concat([result, df['CollegeCityTier']], axis=1)
    result = pd.concat([result, df['English']], axis=1)
    result = pd.concat([result, df['Logical']], axis=1)
    result = pd.concat([result, df['Quant']], axis=1)

    columns = modifyScoreColumns(df)
    for column in columns:
        result = pd.concat([result, df[column]], axis=1)

    result = pd.concat([result, df['conscientiousness']], axis=1)
    result = pd.concat([result, df['agreeableness']], axis=1)
    result = pd.concat([result, df['extraversion']], axis=1)
    result = pd.concat([result, df['nueroticism']], axis=1)
    result = pd.concat([result, df['openess_to_experience']], axis=1)

    result = pd.concat([result, df['Salary']], axis=1)
    result.to_csv("DesignMatrixHim")


In [5]:
main()

### Adding new feature - Salary bins based on 5 - Quantiles of samples

In [6]:
dm = pd.read_csv("DesignMatrixHim", sep=',')

In [7]:
salary_bins = dm['Salary']

In [8]:
bins = pd.qcut(salary_bins, [0, .20, .4, .60, .8, 1.],labels=[1,2,3,4,5])

In [9]:
bins.unique()

[5, 3, 2, 4, 1]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [10]:
dm['salary_bins'] = bins

In [11]:
dm.keys()

Index([u'Unnamed: 0', u'f', u'm', u'cbse', u'icse', u'na', u'state', u'cbse.1',
       u'icse.1', u'na.1', u'state.1', u'B.Tech/B.E.', u'M.Sc. (Tech.)',
       u'M.Tech./M.E.', u'MCA', u'computer', u'mechanical', u'biotechnology',
       u'science', u'applied', u'instrumentation', u'electronics',
       u'electrical', u'application', u'production', u'information',
       u'telecommunications', u'technology', u'civil', u'industrial',
       u'control', u'engineering', u'other', u'Andhra Pradesh', u'Assam',
       u'Bihar', u'Chhattisgarh', u'Delhi', u'Goa', u'Gujarat', u'Haryana',
       u'Himachal Pradesh', u'Jammu and Kashmir', u'Jharkhand', u'Karnataka',
       u'Kerala', u'Madhya Pradesh', u'Maharashtra', u'Meghalaya', u'Orissa',
       u'Punjab', u'Rajasthan', u'Sikkim', u'Tamil Nadu', u'Telangana',
       u'Union Territory', u'Uttar Pradesh', u'Uttarakhand', u'West Bengal',
       u'myDOB', u'12age', u'gradage', u'GraduationYear', u'12graduation',
       u'10percentage', u'12perce

In [12]:
dm['salary_bins']
dm.to_csv("designMatrixHim")

In [13]:
dm = pd.read_csv("designMatrixHim")
bins = dm['salary_bins'].values
np.unique(bins, return_index = True, return_counts = True)


(array([1, 2, 3, 4, 5]),
 array([18,  4,  2,  7,  0]),
 array([1012,  635,  928,  671,  752]))

### Setting up Pipeline and Grid Search for Regression Problem

In [103]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

In [104]:
inputFrame = pd.read_csv("DesignMatrix")
# print inputFrame.tail()
input = inputFrame.as_matrix()
X = input[:,1:-1]
Y = input[:,-1:]
'''
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =500, random_state=5)
X_scaled = scale(X)
scaler = StandardScaler().fit(X)
X_trans = scaler.transform(X)
'''

#### Custom Ridge class custom score function based on percentage error:


In [105]:
class Custom_Ridge(Ridge):
    def __init__(self, alpha=1.0, fit_intercept=True, normalize=False,
                 copy_X=True, max_iter=None, tol=1e-3, solver="auto",
                 random_state=None):
        super(Custom_Ridge, self).__init__(alpha=alpha, fit_intercept=fit_intercept,
                                    normalize=normalize, copy_X=copy_X,
                                    max_iter=max_iter, tol=tol, solver=solver,
                                    random_state=random_state)
        
    def score(self, X, y, sample_weight=None):
        y_t = y
        y_p = self.predict(X)
        r,c = y_t.shape
        J = y_p - y_t
        loss = np.divide(J,y_t)
        meanLoss = np.sum(np.square(loss))/r
        return meanLoss
        

#### Pipeline and gridsearch for Custom Ridge

In [119]:
pipe = Pipeline([
    ('preprocessing', StandardScaler()),
    ('custom_ridge', Custom_Ridge()),
])

parameters = {
    'custom_ridge__alpha': (0.00001, 0.001, 0.001, 0.01, 0.1, 1.0, 10, 100, 1000, 100000 ),
    'custom_ridge__max_iter': (10, 100, 1000, 10000),
}

grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)

In [114]:
gr = grid_search.fit(X, Y)

[Parallel(n_jobs=-1)]: Done  49 tasks       | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    2.5s finished


Fitting 3 folds for each of 40 candidates, totalling 120 fits


In [115]:
gr.best_params_ 

{'custom_ridge__alpha': 1.0, 'custom_ridge__max_iter': 10}

In [116]:
gr.best_score_

9.3267473784919925

In [117]:
gr.grid_scores_

[mean: 9.32172, std: 12.27391, params: {'custom_ridge__alpha': 1e-05, 'custom_ridge__max_iter': 10},
 mean: 9.32172, std: 12.27391, params: {'custom_ridge__alpha': 1e-05, 'custom_ridge__max_iter': 100},
 mean: 9.32172, std: 12.27391, params: {'custom_ridge__alpha': 1e-05, 'custom_ridge__max_iter': 1000},
 mean: 9.32172, std: 12.27391, params: {'custom_ridge__alpha': 1e-05, 'custom_ridge__max_iter': 10000},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_ridge__max_iter': 10},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_ridge__max_iter': 100},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_ridge__max_iter': 1000},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_ridge__max_iter': 10000},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_ridge__max_iter': 10},
 mean: 9.32173, std: 12.27392, params: {'custom_ridge__alpha': 0.001, 'custom_r

In [95]:
pipe.set_params(preprocessing__with_mean=True,preprocessing__with_std=True,custom_ridge__alpha=0.0000001).fit(X,Y)

Pipeline(steps=[('preprocessing', StandardScaler(copy=True, with_mean=True, with_std=True)), ('custom_ridge', Custom_Ridge(alpha=1e-07, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=None, solver='auto', tol=0.001))])

In [96]:
prediction = pipe.predict(X)

In [97]:
pipe.score(X,Y)

0.60919032331739598

### Experiments with sklearn Ridge and pipeline

In [102]:
pipe = Pipeline([
    ('preprocessing', StandardScaler()),
    ('ridge', Ridge()),
])

pipe.set_params(preprocessing__with_mean=True,preprocessing__with_std=True,ridge__alpha=10).fit(X,Y)

prediction = pipe.predict(X)

pipe.score(X,Y)

0.17652324704301658

### Old code

In [100]:
test_loss_list = []
train_loss_list = []
#Ridge Regression
print "Starting ridge Regression"
#for i in range(-10,10):
threshold = 10000
alpha = 0.00000001;
clf = Ridge(alpha=alpha)
clf.fit(X_train, y_train)
ytestpredict = clf.predict(X_test)
ytrainpredict = clf.predict(X_train)
# print y_test.shape
# print ytestpredict.shape
# print y_train.shape
# print ytrainpredict.shape
#test_loss = compute_square_loss(ytestpredict,y_test,threshold)
train_loss = compute_square_loss(ytrainpredict,y_train,threshold)
#test_loss_list.append(test_loss)
#train_loss_list.append(train_loss)
print "train_loss :" , train_loss

Starting ridge Regression
train_loss : 0.58335643944


In [31]:
def feature_normalization(train, test):
    """Rescale the data so that each feature in the training set is in
    the interval [0,1], and apply the same transformations to the test
    set, using the statistics computed on the training set.
    Args:
        train - training set, a 2D numpy array of size (num_instances, num_features)
        test  - test set, a 2D numpy array of size (num_instances, num_features)
    Returns:
        train_normalized - training set after normalization
        test_normalized  - test set after normalization
    """
    #print "inside feature_normalization"
    #print train
    #print test
    nptrain = np.array(train, dtype=np.float64)
    nptest = np.array(test, dtype=np.float64)
    #print nptrain
    r,c = nptrain.shape
    #print r
    #print c
    maxvalues = []
    minvalues = []
    for i in range(0,c,1):
        #print "%d" %(i)
        vslice = nptrain[:,i:i+1]
        #print vslice.shape
        vslicemax = vslice.max()
        vslicemin = vslice.min()
        #print "vslicemax is %d" %(vslicemax)
        #print "vslicemin is %d" %(vslicemin)
        maxvalues.append(vslicemax-vslicemin)
        minvalues.append(vslicemin)

    #print "Outside for loop"
    npmax = np.array(maxvalues)
    npmin = np.array(minvalues)


    rows= len(npmax)
    # print "inside normalisation"
    # print rows
    for i in range(0,rows):
        if (npmax[i] - npmin[i] == 0):
            npmax[i] = npmin[i] + 1


    train_normalized =  np.array((nptrain - npmin) / (npmax - npmin), dtype=np.float64)
    #print train_normalized
    test_normalized =  np.array((nptest - npmin) / (npmax - npmin), dtype=np.float64)
    #print test_normalized
    return train_normalized,test_normalized


def compute_square_loss(y1, y2, threshold):
    r,c = y1.shape
    J = y1 - y2
    loss = np.divide(J,y2)
    meanLoss = np.sum(np.square(loss))/r
    return meanLoss



In [36]:
inputFrame = pd.read_csv("DesignMatrix")
# print inputFrame.tail()
input = inputFrame.as_matrix()
X = input[:,1:-1]
Y = input[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =500, random_state=5)

Xnan = np.isnan(X)
np.isnan(Y)

r,c = X.shape
for i in range(0,r):
    for j in range(0,c):
        if Xnan[i,j] == True:
            print i,j

X_train, X_test =  feature_normalization(X_train, X_test)

Xnan = np.isnan(X_train)
r,c = X_train.shape
for i in range(0,r):
    for j in range(0,c):
        if Xnan[i,j] == True:
            print i,j

Xnan = np.isnan(X_test)
r,c = X_test.shape
for i in range(0,r):
    for j in range(0,c):
        if Xnan[i,j] == True:
            print i,j


test_loss_list = []
train_loss_list = []
#Ridge Regression
print "Starting ridge Regression"
for i in range(-10,10):
    threshold = 10000
    alpha = 10**i;
    clf = Ridge(alpha=alpha)
    clf.fit(X_train, y_train)
    ytestpredict = clf.predict(X_test)
    ytrainpredict = clf.predict(X_train)
    # print y_test.shape
    # print ytestpredict.shape
    # print y_train.shape
    # print ytrainpredict.shape
    test_loss = compute_square_loss(ytestpredict,y_test,threshold)
    train_loss = compute_square_loss(ytrainpredict,y_train,threshold)
    test_loss_list.append(test_loss)
    train_loss_list.append(train_loss)
    print "test_loss :", test_loss , "  train_loss :" , train_loss
    #print i, compute_square_loss(ytestpredict,y_test,threshold), compute_square_loss(ytrainpredict,y_train,threshold)

#Lasso Regression
print "Starting Lasso Regression"
for i in range(-10,10):
    threshold = 10000
    alpha = 10**i;
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(X_train,y_train)
    ytestpredict = clf.predict(X_test)
    ytrainpredict = clf.predict(X_train)
    ytestpredict = np.reshape(ytestpredict, (len(ytestpredict), 1))
    ytrainpredict = np.reshape(ytrainpredict, (len(ytrainpredict), 1))
    # print y_test.shape
    # print ytestpredict.shape
    # print y_train.shape
    # print ytrainpredict.shape
    #print i, compute_square_loss(y_test,ytestpredict,threshold), compute_square_loss(y_train,ytrainpredict,threshold)


Starting ridge Regression
test_loss : 0.804499625793   train_loss : 0.583356438483
test_loss : 0.804499733815   train_loss : 0.583356429766
test_loss : 0.804499611794   train_loss : 0.583356342599
test_loss : 0.804498259836   train_loss : 0.583355470975
test_loss : 0.804484733716   train_loss : 0.583346760071
test_loss : 0.804350225247   train_loss : 0.583260181244
test_loss : 0.803077139363   train_loss : 0.582444944988
test_loss : 0.795183252894   train_loss : 0.5776424936
test_loss : 0.78806764021   train_loss : 0.57568124265
test_loss : 0.785682293743   train_loss : 0.578323832407
test_loss : 0.77186867874   train_loss : 0.577975216301
test_loss : 0.742889145366   train_loss : 0.580315118563
test_loss : 0.719786204417   train_loss : 0.605873380806
test_loss : 0.755010803196   train_loss : 0.678996156912
test_loss : 0.83973432827   train_loss : 0.779389621928
test_loss : 0.880917459544   train_loss : 0.820430452048
test_loss : 0.886779049288   train_loss : 0.826118732459
test_loss :